<a href="https://colab.research.google.com/github/CasualMathEnjoyer/01RAD_Ex09_HW/blob/main/code/01RAD_Ex09_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD  Exercise 09 - Homework

Authors: Your names here  
Date: 2024-12-03  

---

## Task Description

The dataset is based on the **House Sales in King County, USA** dataset, which can be found, for example, on kaggle.com or in the `moderndive` library under the name `house_prices`. The original dataset contains house sale prices in the King County area, which includes Seattle, and the data was collected between May 2014 and May 2015. For our purposes, several variables have been removed, and the dataset has been significantly reduced and slightly modified.

The dataset has already been split into three parts and modified, all of which will be used progressively throughout this assignment.

---

## Variables Description

The dataset contains the following 18 variables, and our goal is to explore the influence of 12 of them on the target variable `price`.

| Feature         | Description                                           |
|------------------|-------------------------------------------------------|
| `id`            | Unique identifier for a house                         |
| `price`         | Sale price (prediction target)                        |
| `bedrooms`      | Number of bedrooms                                    |
| `bathrooms`     | Number of bathrooms                                   |
| `sqft_living`   | Square footage of the home                            |
| `sqft_lot`      | Square footage of the lot                             |
| `floors`        | Total number of floors (levels) in the house          |
| `waterfront`    | Whether the house has a waterfront view               |
| `view`          | Number of times the house has been viewed             |
| `condition`     | Overall condition of the house                        |
| `grade`         | Overall grade given to the housing unit               |
| `sqft_above`    | Square footage of the house apart from the basement   |
| `sqft_basement` | Square footage of the basement                        |
| `yr_built`      | Year the house was built                              |
| `yr_renovated`  | Year when the house was renovated                     |
| `sqft_living15` | Living room area in 2015 (after renovations)          |
| `sqft_lot15`    | Lot size in 2015 (after renovations)                  |
| `split`         | Splitting variable with train, test, and validation samples |

---


In [ ]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD_2024_house.csv"
house_rad = pd.read_csv(url)

# Display the first few rows of the dataset
house_rad.head()



---

## Exploratory and Graphical Analysis

### Question 1

Verify the dimensions of the dataset, the types of individual variables, and summarize the basic descriptive statistics of all variables. Plot a histogram and a density estimate for the target variable `price`. Can anything be inferred for future analysis?

---

### Question 2

Are all variables usable for analysis and prediction of house prices? If the data contains missing values (or strange or nonsensical observations), can they be imputed (corrected), or must they be removed from the dataset?

---

### Question 3

For the selected variables (`price`, `sqft_living`, `grade`, `yr_built`), verify whether the split into train, test, and validation datasets was random. That is, do these variables have approximately the same distributions across the train, test, and validation groups?

---


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math

# Question 1


In [ ]:
# Verify the dimensions of the dataset
print("\nDimensions of the dataset:", house_rad.shape)

# Data types of individual variables
print("\nData types of variables:")
print(house_rad.dtypes)

# Summarize basic descriptive statistics of all variables
print("\nDescriptive statistics:")
print(house_rad.describe(include='all'))


**Potential Outliers:**

  - Bathrooms: The maximum value (275) is abnormally high.
  - Grade: The maximum value (232) is unusually high compared to the 75th percentile (8), suggesting extreme outliers.
  - Sqft_living and Sqft_lot: The large maximum values (33,600 and 641,203, respectively) compared to their means and 75th percentiles suggest outliers.
  - Sqft_living and Sqft_lot: Small minimum value (105 and 12) might be an outlier.

**Missing or Sparse Data:**

  - Waterfront: The mean value is 0.021, indicating that only about 2.1% of houses have a waterfront view.
  - Yr_renovated: Most houses likely have not been renovated, as the median value is 0, which might represent missing or unrenovated properties.
  - There aren't any NAN values in the dataset

**Notes and observations**

  - Year Built: The range (1900-2015) suggests a mix of old and modern homes.

---



In [ ]:
# Plot histograms of the specified columns
house_rad.hist(bins=30, figsize=(20, 20), grid=False, edgecolor='black', alpha=0.7)
plt.suptitle("Filtered Data", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

plt.tight_layout()
plt.show()

In [ ]:
# Plot histogram and density estimate for the target variable price
plt.figure(figsize=(12, 6))
house_rad['price'].hist(bins=100, alpha=0.7, label='Histogram', density=True)
house_rad['price'].plot(kind='kde', label='Density Estimate', color='red')
plt.title('Distribution of Sale Price')
plt.xlabel('Price')
plt.ylabel('Frequency/Density')
plt.legend()
plt.show()

The histogram suggest that the price distribution is right-skewed, with most prices concentrated in the lower range but a long tail extending to very high prices.

Většina naměřených dat v hodnotách kolem milionu dolarů, zatímco pár měření výrazně přes milion - 4, 6, 7, přes 8.

# Question 2


In [ ]:
# Check for missing values in each column
missing_values = house_rad.isnull().sum()

# Display columns with missing values
columns_with_nans = missing_values[missing_values > 0]

print("Columns with missing values and their counts:")
print(columns_with_nans)


There aren't any variables with missing values

In [ ]:
# Select a subset of columns to visualize (price and continuous variables)
columns_to_plot = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                   'floors', 'view', 'sqft_above', 'sqft_basement',
                   'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

# Create pairplot
sns.pairplot(house_rad[columns_to_plot], diag_kind='hist', height=2.5)
plt.show()

In [ ]:
# # Plot histograms for all columns
# house_rad[columns_to_plot].hist(bins=30, figsize=(15, 10), grid=False, edgecolor='black', alpha=0.7)
# plt.suptitle("All Data", fontsize=16)
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()

### Visualize potential outliers:

In [ ]:
numeric_columns = house_rad.select_dtypes(include=['number']).columns[2:]
num_plots = len(numeric_columns)

# Define the grid size (e.g., 3 columns per row)
ncols = 4
nrows = math.ceil(num_plots / ncols)

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 5))
axes = axes.flatten()  # Flatten in case of multiple rows

# Loop through each numeric column and plot a boxplot
for ax, col in zip(axes, numeric_columns):
    sns.boxplot(x=house_rad[col], ax=ax)
    ax.set_title(f"Boxplot of {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.7)

# Remove any unused subplots
for ax in axes[num_plots:]:
    fig.delaxes(ax)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the grid of boxplots
plt.show()

In [ ]:
# Create a figure with 1 row and 2 columns of subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 6))  # Adjust figsize as needed

# First Scatter Plot: Price vs. Living Area
axs[0].scatter(house_rad['price'], house_rad['sqft_living'], alpha=0.5, color='blue')
axs[0].set_title("Scatter Plot: Price vs. Living Area", fontsize=16)
axs[0].set_xlabel("Price", fontsize=12)
axs[0].set_ylabel("Living Area (sqft)", fontsize=12)
axs[0].grid(True, alpha=0.3)

# Second Scatter Plot: Price vs. Lot Area
axs[1].scatter(house_rad['price'], house_rad['sqft_lot'], alpha=0.5, color='blue')
axs[1].set_title("Scatter Plot: Price vs. Lot Area", fontsize=16)
axs[1].set_xlabel("Price", fontsize=12)
axs[1].set_ylabel("Lot Area (sqft)", fontsize=12)
axs[1].grid(True, alpha=0.3)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()

Zdá se, že hodnoty s vysokou cenou mohou být zatíženy chybou (panem Francem), neboť se nezdá že by reflektovaly trend a pro obrovské ceny mají velmi malou Living Area, stejně jako poměrně nízké ceny v poměru s obrovksými Living Area



---


Vzhledem k tomu, že nevíme jaké jsou správné hodnoty, navrhruji nevyhovující hodnoty vyhodit.
Např jak zmíněno výše: posledních několik hodnot u price jsou nepoměrně velké k ostatním a nedávají smysl, stejně jako hodnoty s nesmyslně velkým Living Area za neadekvátní ceny.

Proměnná grade ná zřejmě také outliera, neboť se vyskytuje jedna hodnota 232 která narušuje všechny ostatní statistiky

Proměnná Waterfall view by mohla činit problémy neboť je u většiny 0, ale je to dalši typ kategorické proměnné a je možné pro ni dělat regresi zvlášť?

In [ ]:
# Visualization for the data with a basic outlier elimination
# Filter data to remove outliers

# KATKA
# filtered_house_data = house_rad[
#     (house_rad['bathrooms'] < 5) &
#     (house_rad['sqft_living'] < 30000) &
#     (house_rad['sqft_lot'] < 200000) &
#     (house_rad['sqft_living15'] < 30000) &
#     (house_rad['sqft_lot15'] < 200000)
# ]

# FILIP:
data = house_rad
filtered_house_data = data[
    (data['bathrooms'] < 5) &
    (data['sqft_living'] < 10000) &
    (data['sqft_lot'] < 30000) &
    (data['sqft_living15'] < 10000) &
    (data['sqft_lot15'] < 30000) &
    (data['price'] < 4000000) &
    (data['grade'] < 20)
]

# Plot histograms of the specified columns
filtered_house_data[columns_to_plot].hist(bins=30, figsize=(20, 20), grid=False, edgecolor='black', alpha=0.7)
plt.suptitle("Filtered Data", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit title
plt.show()

summary_stats = filtered_house_data.describe(include='all')
summary_stats

Odebral jsem některé hodnoty podle plotů jednotlivých proměnných a podle histogramů. Postupoval jsem podle nějakých logických úsudků - living area větší než 10000 ft^2 neboli 929 m^2 nedává smysl, tak jsem to tímto způsobem omezil. Stejně jako cena nemovitosti - v scatter plotu price ku living space jsou byty s velmi malými living space řádově o několik milionu dražší, což nedává z logiky věci smysl, a proto jsem ji omezil (je otázka jesli mohu omezit cenu když je to naše target proměnná). Obdobně grade dává smysl 0-10, vyskytuje se ale i hodnota 13. Lot area je také - při velmi nízkých cenách prodeje jsou velmi obrovské pozemky, v řádu 2800 m^2.

In [ ]:
# Assuming 'filtered_house_data' is your DataFrame containing numeric columns
numeric_columns = filtered_house_data.select_dtypes(include=['number']).columns[2:]
num_plots = len(numeric_columns)

# Define the grid size (e.g., 3 columns per row)
ncols = 4
nrows = math.ceil(num_plots / ncols)

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 5))
axes = axes.flatten()  # Flatten in case of multiple rows

# Loop through each numeric column and plot a boxplot
for ax, col in zip(axes, numeric_columns):
    sns.boxplot(x=filtered_house_data[col], ax=ax)
    ax.set_title(f"Boxplot of {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.7)

# Remove any unused subplots
for ax in axes[num_plots:]:
    fig.delaxes(ax)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the grid of boxplots
plt.show()

# Question 3

FIRST LOOK AT THE DATA BASED ON SPLIT

In [ ]:
figsize = (12, 1.2 * len(house_rad['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(house_rad, x='price', y='split', inner='box', color="orange")
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
figsize = (12, 1.2 * len(house_rad['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(house_rad, x='sqft_living', y='split', inner='box', color='green')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
figsize = (12, 1.2 * len(house_rad['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(house_rad, x='grade', y='split', inner='box', color='grey')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
figsize = (12, 1.2 * len(house_rad['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(house_rad, x='yr_built', y='split', inner='box', color="cyan")
sns.despine(top=True, right=True, bottom=True, left=True)

DENSITY ESTIMATES

In [ ]:
selected_variables = ['price', 'sqft_living', 'grade', 'yr_built']

# Create a 2x2 grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))  # Adjust figsize as needed
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

# Loop through each variable and corresponding subplot axis
for ax, variable in zip(axes, selected_variables):
    sns.kdeplot(
        data=data,
        x=variable,
        hue='split',
        fill=True,
        common_norm=False,
        alpha=0.5,
        ax=ax
    )
    # Set plot title and labels
    ax.set_title(f"Distribution of {variable.replace('_', ' ').title()} Across Splits", fontsize=14)
    ax.set_xlabel(variable.replace('_', ' ').title(), fontsize=12)
    ax.set_ylabel("Density", fontsize=12)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the grid of density plots
plt.show()

Visually, we can see there might be a problem with variable grade.

In [ ]:
from scipy.stats import ks_2samp, f_oneway, kruskal

# Group by split
train_data = data[data['split'] == 'train']
test_data = data[data['split'] == 'test']
validation_data = data[data['split'] == 'validation']

for variable in selected_variables:
    print(f"\n=== Analysis for {variable} ===")

    # Kolmogorov-Smirnov Test: Train vs Test
    ks_train_test = ks_2samp(train_data[variable], test_data[variable])
    print(f"KS Test (Train vs Test): Statistic={ks_train_test.statistic:.4f}, p-value={ks_train_test.pvalue:.4f}")

    # Kolmogorov-Smirnov Test: Train vs Validation
    ks_train_validation = ks_2samp(train_data[variable], validation_data[variable])
    print(f"KS Test (Train vs Validation): Statistic={ks_train_validation.statistic:.4f}, p-value={ks_train_validation.pvalue:.4f}")

    # Kolmogorov-Smirnov Test: Test vs Validation
    ks_test_validation = ks_2samp(test_data[variable], validation_data[variable])
    print(f"KS Test (Test vs Validation): Statistic={ks_test_validation.statistic:.4f}, p-value={ks_test_validation.pvalue:.4f}")

    # ANOVA
    anova_result = f_oneway(train_data[variable], test_data[variable], validation_data[variable])
    print(f"ANOVA: F-statistic={anova_result.statistic:.4f}, p-value={anova_result.pvalue:.4f}")

    # Kruskal-Wallis Test
    kruskal_result = kruskal(train_data[variable], test_data[variable], validation_data[variable])
    print(f"Kruskal-Wallis Test: H-statistic={kruskal_result.statistic:.4f}, p-value={kruskal_result.pvalue:.4f}")

Pouze u analýzy **'grade'** nám p-value u Anovy vyšla velmi nízká, ale i tak ne menší než 0.05, takže bychom hypotézu, že jsou střední hodnoty tří rozdělení odlišné nezamítly. I když na obrázku PDFs je vidět, že split u grade není tak podobný jako u jiných proměnných



# Linear Model (Use Only Training Data, i.e., split == "train")

### Question 4

Calculate the correlations between the regressors and visualize them. Also, compute the condition number (Kappa) and the variance inflation factor (VIF). If multicollinearity is present, decide which variables to use and justify your choices.

---

### Question 5

Using only the training data (split == "train") and all selected variables, find a suitable linear regression model that best predicts the target variable `price`, i.e., minimizes the mean squared error (MSE). Compare the VIF and Kappa values of the final model to those of the original regressor matrix.

---

### Question 6

For your selected model from the previous question, calculate the relevant influence measures. Provide the `id` for the top 20 observations with the highest values of DIFF, the highest leverage (hat values), and the highest Cook’s distance (i.e., 3 sets of 20 values). Which observations do you consider influential or outliers?

---

### Question 7

Validate the model graphically using residual plots (Residual vs. Fitted, QQ-plot, Cook’s distance, leverage, etc.). Based on this and the previous question, did you identify any suspicious observations in the data that might have resulted from dataset adjustments? Would you recommend removing these observations from the data?

---


# Question 4

In [ ]:
# Select only numeric columns
numeric_columns = house_rad.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap')
plt.show()



Correlation:
- price and sqft_living, sqft_above, sft_living14
- price view, waterfront, floors, bedrooms, yr_renovated
- svarious sqft correlations (as expected)

Negative correlation:
- year build vs year renovated (as expected)

# Question 5


In [ ]:
def model_Q5(data, selected_features, split):
    print("Model for data split:", split)
    split_data = data[data['split'] == split]
    X = split_data[selected_features]
    y = split_data['price']

    # Handle missing values
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

    # # Create interaction terms
    # X['sqft_living_grade'] = X['sqft_living'] * X['grade']
    # X['bathrooms_bedrooms'] = X['bathrooms'] * X['bedrooms']

    # Add a constant for statsmodels regression
    X_with_const = sm.add_constant(X)

    # Fit the regression model
    model = sm.OLS(y, X_with_const).fit()

    # Print the summary
    print(model.summary())

    # Calculate Mean Squared Error (MSE)
    y_pred = model.predict(X_with_const)
    mse = ((y - y_pred) ** 2).mean()

    # Calculate kappa (condition number of the design matrix)
    kappa = np.linalg.cond(X_with_const)

    # Calculate VIF for each feature
    vif_data = pd.DataFrame({
        "Variable": X_with_const.columns,
        "VIF": [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
    })

    # Display results
    print("\nVariance Inflation Factors (VIF):")
    print(vif_data)
    print(f"\nMean Squared Error (MSE): {mse}")
    print(f"Kappa (Condition Number): {kappa}")
    print('\n')


In [ ]:
selected_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                     'floors', 'waterfront', 'view', 'condition',
                     'grade', 'sqft_above', 'sqft_basement',
                     'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']
selected_features = ['sqft_living',
                     'floors', 'view',
                     'sqft_basement',
                     'yr_renovated']


model_Q5(house_rad, selected_features, split='train')


# Robust Approach

### Question 8

If you decided to remove any observations from the dataset, work with the filtered dataset, retrain the model from Question 5, and calculate the $R^2$ statistic and MSE on both the training and test data (split == "test").

---

### Question 9

Estimate the regression coefficients using a robust method, such as Total Least Squares (TLS), on both the filtered dataset (after removing any observations, if applicable) and the original full dataset. Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?

---
### Question 10

Explore additional robust regression approaches (e.g., Huber regression, quantile regression, or M-estimators) to estimate the regression coefficients on the filtered and full datasets. Compare the results across these methods, discussing the strengths and limitations of each approach in the context of predicting house prices in the King County area. How do these robust methods handle potential outliers or influential observations in the data?

---
### Question 11

Select the final model and compare the MSE and $R^2$ on the training, test, and validation datasets. What do these values suggest about the quality of the model and potential overfitting? Is your model suitable for predicting house prices in the King County area? If so, does this prediction have any limitations?



# Question 8

In [ ]:
# Display rows with extreme values for bathrooms


outliers = house_rad[house_rad['bathrooms'] > 5]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_living'] < house_rad['sqft_above']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_living'] < house_rad['sqft_basement']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_above']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_basement']]
print(outliers.shape)

# outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_living']]  # doesn't necesarily logically need to be an outlier - building can be vertical
# print(outliers.shape)

# outliers = house_rad[house_rad['sqft_lot15'] < house_rad['sqft_living15']]
# print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < 100]
print(outliers.shape)

outliers = house_rad[(house_rad['sqft_living'] < 500) & (house_rad['bathrooms'] > 2)]  # not added condition, just seemed weird
print(outliers.shape)

# Define conditions
conditions = (
    (house_rad['bathrooms'] <= 5)
    & (house_rad['sqft_lot'] > 100)
    & (house_rad['sqft_living'] >= house_rad['sqft_above'])
    & (house_rad['sqft_living'] >= house_rad['sqft_basement'])
    & (house_rad['sqft_lot'] >= house_rad['sqft_above'])
    & (house_rad['sqft_lot'] >= house_rad['sqft_basement'])
    # & (house_rad['sqft_lot'] >= house_rad['sqft_living'])
    # & (house_rad['sqft_lot15'] >= house_rad['sqft_living15'])
)

# Apply the conditions to filter the dataset
filtered_data = house_rad[conditions]

# Print dataset sizes
print(f"Original dataset size: {house_rad.shape}")
print(f"Filtered dataset size after manual removal: {filtered_data.shape}")



In [ ]:
# test on model from question 5
model_Q5(filtered_data, selected_features, split="train")
model_Q5(filtered_data, selected_features, split="test")

# Question 9


In [ ]:
# ON UNFILTERED DATA

X = house_rad[selected_features]
y = house_rad['price']

X_with_const = sm.add_constant(X) # Add constant for intercept

# Fit Robust Linear Model
rlm_model_all = sm.RLM(y, X_with_const)
rlm_results_all = rlm_model_all.fit()
print(rlm_results_all.summary())

In [ ]:
# ON FILTERED DATA

X = filtered_data[selected_features]
y = filtered_data['price']

X_with_const = sm.add_constant(X) # Add constant for intercept

# Fit Robust Linear Model
rlm_model_filtered = sm.RLM(y, X_with_const)
rlm_results_filtered = rlm_model_filtered.fit()
print(rlm_results_filtered.summary())

*TODO:
Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?*

# Question 10


In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
def huber_model(data):
    # Prepare data
    X = data[selected_features]
    y = data['price']

    # Fit Huber Regressor
    huber = HuberRegressor()
    huber.fit(X, y)

    # Get coefficients and intercept
    coefficients = huber.coef_
    intercept = huber.intercept_

    # Predict and evaluate
    y_pred = huber.predict(X)
    mse = mean_squared_error(y, y_pred)

    print("Coefficients:", coefficients)
    print("Intercept:", intercept)
    print("MSE:", mse)

print("All data")
huber_model(house_rad)

print('\n')
print("Filtered data")
huber_model(filtered_data)




In [ ]:
def quantile_model(data, q=0.5):
    # Prepare data
    X = data[selected_features].fillna(0)
    y = data['price'].fillna(0)

    # Add constant for intercept
    X_with_const = sm.add_constant(X)

    # Fit Quantile Regression
    quantile_reg = sm.QuantReg(y, X_with_const)
    quantile_results = quantile_reg.fit(q=q)

    print(quantile_results.summary())

    # Predict and evaluate
    y_pred = quantile_results.predict(X_with_const)
    mse = mean_squared_error(y, y_pred)

    print(f"Quantile (q={q}) Regression Results")
    print("Coefficients:", quantile_results.params.values)
    print("Intercept:", quantile_results.params['const'])
    print("MSE:", mse)

print("All data")
quantile_model(house_rad, q=0.5)

print('\n')
print("Filtered data")
quantile_model(filtered_data, q=0.5)


In [ ]:
def m_estimator_model(data):
    # Prepare data
    X = data[selected_features].fillna(0)
    y = data['price'].fillna(0)

    # Add constant for intercept
    X_with_const = sm.add_constant(X)

    # Fit Robust Linear Model (M-Estimator)
    m_estimator = sm.RLM(y, X_with_const)
    m_estimator_results = m_estimator.fit()

    print(m_estimator_results.summary())

    # Predict and evaluate
    y_pred = m_estimator_results.predict(X_with_const)
    mse = mean_squared_error(y, y_pred)

    print("M-Estimator Regression Results")
    print("Coefficients:", m_estimator_results.params.values)
    print("Intercept:", m_estimator_results.params['const'])
    print("MSE:", mse)

print("All data")
m_estimator_model(house_rad)

print('\n')
print("Filtered data")
m_estimator_model(filtered_data)


*TODO Compare the results across these methods, discussing the strengths and limitations of each approach in the context of predicting house prices in the King County area. How do these robust methods handle potential outliers or influential observations in the data?*

# Question 11


---

# Voluntary part: Machine Learning Approach


### Question 12
Apply machine learning-based linear regression methods, such as Ridge Regression, Lasso, or Elastic Net, to the dataset. Use the train-test split to evaluate model performance and focus on feature selection. Identify the most relevant features based on these methods and compare how the selected features impact the model's predictive performance. Discuss how regularization affects feature selection and the trade-offs between bias and variance in the context of house price prediction. Additionally, evaluate the stability of selected features across different methods and provide recommendations for choosing the optimal feature set.
